# LangChain-based Course Planning System

This notebook implements an intelligent course planning system that:
- Analyzes whether a topic is broad or narrow
- Generates a single course for narrow topics
- Splits broad topics into multiple logically structured courses (max 8)
- Orders courses from beginner to advanced
- Ensures no overlap between courses
- Returns structured output using Pydantic models

## Install Required Packages

In [26]:
%pip install langchain langchain-ollama pydantic python-dotenv -q

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.0 -> 26.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


## Import Dependencies

In [27]:
from typing import List
from pydantic import BaseModel, Field, field_validator
from langchain_ollama import ChatOllama
from langchain_core.prompts import ChatPromptTemplate

## Define Pydantic Models

These models ensure strict validation of the output structure.

In [28]:
class Course(BaseModel):
    """Represents a single course in the learning path."""
    course_name: str = Field(..., description="Name of the course")
    description: str = Field(..., description="Detailed description of the course")
    difficulty: str = Field(..., description="Difficulty level: Beginner, Intermediate, or Advanced")
    prerequisites: List[str] = Field(default_factory=list, description="List of prerequisite courses")
    
    @field_validator('difficulty')
    @classmethod
    def validate_difficulty(cls, v: str) -> str:
        """Ensure difficulty is one of the allowed values."""
        allowed = ['Beginner', 'Intermediate', 'Advanced']
        if v not in allowed:
            raise ValueError(f'Difficulty must be one of {allowed}')
        return v


class CoursePlan(BaseModel):
    """Represents a complete course plan with one or more courses."""
    is_broad: bool = Field(..., description="Whether the topic is broad and requires multiple courses")
    total_courses: int = Field(..., description="Total number of courses in the plan")
    courses: List[Course] = Field(..., description="List of courses in the plan")
    
    @field_validator('total_courses')
    @classmethod
    def validate_total_courses(cls, v: int, info) -> int:
        """Ensure total_courses matches the length of courses list."""
        if 'courses' in info.data:
            actual_count = len(info.data['courses'])
            if v != actual_count:
                raise ValueError(f'total_courses ({v}) must equal the number of courses ({actual_count})')
        return v
    
    @field_validator('courses')
    @classmethod
    def validate_max_courses(cls, v: List[Course]) -> List[Course]:
        """Ensure maximum 8 courses."""
        if len(v) > 8:
            raise ValueError('Maximum 8 courses allowed')
        if len(v) < 1:
            raise ValueError('At least 1 course is required')
        return v

## Create Course Planning System

This class uses LangChain with structured output to generate course plans.

In [29]:
class CoursePlanner:
    """LangChain-based course planning system."""
    
    def __init__(self, model_name: str = "phi3:mini", temperature: float = 0.7):
        """
        Initialize the course planner.
        
        Args:
            model_name: Name of the Ollama model to use
            temperature: Temperature for response generation (0.0-1.0)
        """
        self.llm = ChatOllama(model=model_name, temperature=temperature)
        self.structured_llm = self.llm.with_structured_output(CoursePlan)
        self.prompt = self._create_prompt()
        self.chain = self.prompt | self.structured_llm
    
    def _create_prompt(self) -> ChatPromptTemplate:
        """Create the prompt template for course planning."""
        system_message = """You are an expert curriculum designer and educational consultant.

Your task is to analyze a given topic and create a structured course plan.

ANALYSIS RULES:
1. Determine if the topic is BROAD or NARROW:
   - NARROW: A specific, focused topic that can be covered in a single comprehensive course
     Examples: "Introduction to Python Lists", "CSS Flexbox", "Linear Regression in Machine Learning"
   
   - BROAD: A wide-ranging topic that requires multiple courses to cover comprehensively
     Examples: "Machine Learning", "Web Development", "Data Science", "Artificial Intelligence"

2. For NARROW topics:
   - Create exactly 1 course
   - Set is_broad = false
   - Provide a comprehensive course covering the entire topic
   
3. For BROAD topics:
   - Create 2-8 courses (maximum 8)
   - Set is_broad = true
   - Split the topic into logical, non-overlapping courses
   - Order courses from Beginner → Intermediate → Advanced
   - Ensure each course builds upon previous ones
   - Assign appropriate prerequisites

COURSE STRUCTURE RULES:
- Each course must have: course_name, description, difficulty, prerequisites
- Difficulty MUST be EXACTLY one of these three values: "Beginner", "Intermediate", or "Advanced"
- DO NOT use combinations like "Beginner-Intermediate" or "Intermediate-Advanced"
- DO NOT create custom difficulty levels - use ONLY the three allowed values
- Prerequisites should reference course names from earlier courses in the plan
- First course should typically be "Beginner" with no prerequisites
- Courses should progress logically in difficulty
- Courses must NOT overlap in content
- Each course should cover a distinct subset of the broader topic

OUTPUT RULES:
- total_courses MUST equal the actual number of courses
- Maximum 8 courses allowed
- Do NOT add any extra fields
- Ensure all required fields are present
- difficulty field accepts ONLY: "Beginner", "Intermediate", or "Advanced" - no other values allowed"""

        user_message = """Create a course plan for the following topic:

Course Title: {course_title}
Course Description: {course_description}

Analyze whether this topic is broad or narrow, then create an appropriate course plan."""

        return ChatPromptTemplate.from_messages([
            ("system", system_message),
            ("user", user_message)
        ])
    
    def create_course_plan(self, course_title: str, course_description: str) -> CoursePlan:
        """
        Generate a course plan based on the input.
        
        Args:
            course_title: The title of the topic
            course_description: A description of what should be covered
            
        Returns:
            CoursePlan object with structured course information
        """
        result = self.chain.invoke({
            "course_title": course_title,
            "course_description": course_description
        })
        
        return result
    
    def print_course_plan(self, plan: CoursePlan) -> None:
        """
        Pretty print the course plan.
        
        Args:
            plan: CoursePlan object to display
        """
        print("=" * 80)
        print(f"COURSE PLAN ANALYSIS")
        print("=" * 80)
        print(f"Topic is: {'BROAD' if plan.is_broad else 'NARROW'}")
        print(f"Total Courses: {plan.total_courses}")
        print("=" * 80)
        print()
        
        for idx, course in enumerate(plan.courses, 1):
            print(f"📚 COURSE {idx}: {course.course_name}")
            print(f"   Difficulty: {course.difficulty}")
            print(f"   Description: {course.description}")
            if course.prerequisites:
                print(f"   Prerequisites: {', '.join(course.prerequisites)}")
            else:
                print(f"   Prerequisites: None")
            print()
        print("=" * 80)

## Initialize the Course Planner

In [30]:
planner = CoursePlanner(model_name="phi3:mini", temperature=0.7)

## Example 1: Narrow Topic (Single Course)

In [31]:
# Example of a narrow topic that should result in a single course
narrow_plan = planner.create_course_plan(
    course_title="Introduction to Python Decorators",
    course_description="Learn how to use and create decorators in Python, including function decorators, class decorators, and decorator patterns."
)

planner.print_course_plan(narrow_plan)

COURSE PLAN ANALYSIS
Topic is: NARROW
Total Courses: 1

📚 COURSE 1: Python Decorators Basics
   Difficulty: Beginner
   Description: Dive into the world of Python decorators. Understand what they are and learn how to apply function-level decorators in real-world scenarios.
   Prerequisites: None



## Example 2: Broad Topic (Multiple Courses)

In [32]:
# Example of a broad topic that should result in multiple courses
broad_plan = planner.create_course_plan(
    course_title="Machine Learning",
    course_description="Comprehensive coverage of machine learning concepts, algorithms, and applications including supervised learning, unsupervised learning, deep learning, and practical implementations."
)

planner.print_course_plan(broad_plan)

COURSE PLAN ANALYSIS
Topic is: BROAD
Total Courses: 4

📚 COURSE 1: Introduction to Machine Learning
   Difficulty: Beginner
   Description: An overview of the field of machine learning and its importance in solving complex problems across various industries.
   Prerequisites: None

📚 COURSE 2: Foundations of Supervised Learning
   Difficulty: Intermediate
   Description: This course introduces fundamental concepts such as regression, classification, neural networks with a focus on supervision learning techniques and their applications in real-world scenarios.
   Prerequisites: Introduction to Machine Learning

📚 COURSE 3: Unsupervised Learning Techniques
   Difficulty: Intermediate
   Description: Learn about clustering, dimensionality reduction and density estimation techniques in unsupervised machine learning along with the challenges of model selection.
   Prerequisites: Introduction to Machine Learning

📚 COURSE 4: Deep Dive into Deep Learning
   Difficulty: Advanced
   Description

## Example 3: Another Broad Topic

In [33]:
# Another broad topic example
web_dev_plan = planner.create_course_plan(
    course_title="Full Stack Web Development",
    course_description="Complete guide to becoming a full stack web developer, covering frontend technologies, backend development, databases, deployment, and modern web development practices."
)

planner.print_course_plan(web_dev_plan)

COURSE PLAN ANALYSIS
Topic is: BROAD
Total Courses: 6

📚 COURSE 1: Introduction to Web Development
   Difficulty: Beginner
   Description: A foundational understanding of web development concepts and the technology stack.
   Prerequisites: None

📚 COURSE 2: HTML & CSS Basics
   Difficulty: Beginner
   Description: Learn to create responsive, visually appealing web pages with HTML and CSS.
   Prerequisites: Introduction to Web Development

📚 COURSE 3: Responsive Design Principles
   Difficulty: Intermediate
   Description: Explore the principles of responsive design and learn how to create websites that adapt across various devices.
   Prerequisites: HTML & CSS Basics

📚 COURSE 4: JavaScript Fundamentals
   Difficulty: Intermediate
   Description: Gain a solid foundation in JavaScript to enhance user interaction on the front end.
   Prerequisites: HTML & CSS Basics

📚 COURSE 5: Backend Development with Node.js
   Difficulty: Intermediate
   Description: Dive into server-side programming

## Validate the Output Structure

In [34]:
# Validate the structure of the generated plan
print("Validation Results:")
print("=" * 80)

# Check narrow topic
print(f"\n✓ Narrow topic plan:")
print(f"  - is_broad: {narrow_plan.is_broad} (Expected: False)")
print(f"  - total_courses: {narrow_plan.total_courses} (Expected: 1)")
print(f"  - Actual courses: {len(narrow_plan.courses)}")
print(f"  - Match: {narrow_plan.total_courses == len(narrow_plan.courses)}")

# Check broad topic
print(f"\n✓ Broad topic plan:")
print(f"  - is_broad: {broad_plan.is_broad} (Expected: True)")
print(f"  - total_courses: {broad_plan.total_courses}")
print(f"  - Actual courses: {len(broad_plan.courses)}")
print(f"  - Match: {broad_plan.total_courses == len(broad_plan.courses)}")
print(f"  - Within limit (1-8): {1 <= len(broad_plan.courses) <= 8}")

# Validate difficulties
print(f"\n✓ Difficulty validation:")
valid_difficulties = ['Beginner', 'Intermediate', 'Advanced']
for course in broad_plan.courses:
    is_valid = course.difficulty in valid_difficulties
    print(f"  - {course.course_name}: {course.difficulty} ({'✓' if is_valid else '✗'})")

print("\n" + "=" * 80)

Validation Results:

✓ Narrow topic plan:
  - is_broad: False (Expected: False)
  - total_courses: 1 (Expected: 1)
  - Actual courses: 1
  - Match: True

✓ Broad topic plan:
  - is_broad: True (Expected: True)
  - total_courses: 4
  - Actual courses: 4
  - Match: True
  - Within limit (1-8): True

✓ Difficulty validation:
  - Introduction to Machine Learning: Beginner (✓)
  - Foundations of Supervised Learning: Intermediate (✓)
  - Unsupervised Learning Techniques: Intermediate (✓)
  - Deep Dive into Deep Learning: Advanced (✓)



## Export to JSON

You can export the course plan to JSON format for API integration.

In [35]:
# Convert to JSON
json_output = broad_plan.model_dump_json(indent=2)
print("JSON Output:")
print(json_output)

# You can also convert to dict
dict_output = broad_plan.model_dump()
print("\n\nDictionary Output:")
print(dict_output)

JSON Output:
{
  "is_broad": true,
  "total_courses": 4,
  "courses": [
    {
      "course_name": "Introduction to Machine Learning",
      "description": "An overview of the field of machine learning and its importance in solving complex problems across various industries.",
      "difficulty": "Beginner",
      "prerequisites": []
    },
    {
      "course_name": "Foundations of Supervised Learning",
      "description": "This course introduces fundamental concepts such as regression, classification, neural networks with a focus on supervision learning techniques and their applications in real-world scenarios.",
      "difficulty": "Intermediate",
      "prerequisites": [
        "Introduction to Machine Learning"
      ]
    },
    {
      "course_name": "Unsupervised Learning Techniques",
      "description": "Learn about clustering, dimensionality reduction and density estimation techniques in unsupervised machine learning along with the challenges of model selection.",
      "d

## Interactive Usage

Use this cell to test with your own topics!

In [36]:
# Try your own topic here!
custom_plan = planner.create_course_plan(
    course_title="Your Topic Here",  # Change this
    course_description="Your description here"  # Change this
)

planner.print_course_plan(custom_plan)

COURSE PLAN ANALYSIS
Topic is: NARROW
Total Courses: 1

📚 COURSE 1: Introduction to Python Lists
   Difficulty: Beginner
   Description: This foundational course is designed for beginners interested in learning about one of the core data structures within programming with Python: lists. Students will understand how lists work as versatile containers that can hold a sequence of items, learn various list operations such as indexing, slicing, appending, and extending, explore common mistakes to avoid when working with mutable objects like lists in Python.
   Prerequisites: None



---

## Part 2: Syllabus Generation with User Preferences

This section extends the course planner to:
1. Take a course and gather user preferences
2. Generate a detailed syllabus with modules and topics
3. Validate the syllabus through a two-layer review system

### Define Syllabus Models

In [37]:
class Topic(BaseModel):
    """Represents a single topic within a module."""
    topic_name: str = Field(..., description="Name of the topic")
    short_description: str = Field(..., description="Short description for users (1-2 sentences)")
    detailed_description: str = Field(..., description="Detailed description for AI to understand the scope and content")
    estimated_duration_minutes: int = Field(..., description="Estimated time to complete this topic in minutes", gt=0)


class Module(BaseModel):
    """Represents a module containing multiple topics."""
    module_name: str = Field(..., description="Name of the module")
    module_description: str = Field(..., description="Overview of what this module covers")
    topics: List[Topic] = Field(..., description="List of topics in this module")
    
    @field_validator('topics')
    @classmethod
    def validate_topics(cls, v: List[Topic]) -> List[Topic]:
        """Ensure at least one topic per module."""
        if len(v) < 1:
            raise ValueError('Each module must have at least 1 topic')
        return v


class Syllabus(BaseModel):
    """Represents a complete course syllabus."""
    course_name: str = Field(..., description="Name of the course")
    course_objective: str = Field(..., description="Main objective of the course")
    learning_style: str = Field(..., description="Learning style preferences applied")
    total_modules: int = Field(..., description="Total number of modules")
    modules: List[Module] = Field(..., description="List of modules in the syllabus")
    total_estimated_hours: float = Field(..., description="Total estimated hours to complete the course")
    
    @field_validator('total_modules')
    @classmethod
    def validate_total_modules(cls, v: int, info) -> int:
        """Ensure total_modules matches the length of modules list."""
        if 'modules' in info.data:
            actual_count = len(info.data['modules'])
            if v != actual_count:
                raise ValueError(f'total_modules ({v}) must equal the number of modules ({actual_count})')
        return v
    
    @field_validator('modules')
    @classmethod
    def validate_modules(cls, v: List[Module]) -> List[Module]:
        """Ensure at least one module."""
        if len(v) < 1:
            raise ValueError('Syllabus must have at least 1 module')
        return v


class SyllabusReview(BaseModel):
    """Represents a review of a syllabus by the validation layer."""
    approved: bool = Field(..., description="Whether the syllabus is approved")
    feedback: str = Field(..., description="Feedback or reasons for rejection/approval")
    issues: List[str] = Field(default_factory=list, description="List of specific issues found (if rejected)")

### User Preference Options

In [38]:
class UserPreferences:
    """Handles user preference collection and system prompt generation."""
    
    PREFERENCE_OPTIONS = {
        "1": {
            "name": "Real-world Examples",
            "description": "Course filled with practical, real-world examples and case studies",
            "prompt_addition": "Include extensive real-world examples, case studies, and practical applications for every concept. Show how each topic is used in industry."
        },
        "2": {
            "name": "Theory-Focused",
            "description": "Deep theoretical understanding with mathematical foundations",
            "prompt_addition": "Focus on theoretical foundations, mathematical principles, and academic rigor. Include proofs and formal definitions where applicable."
        },
        "3": {
            "name": "Project-Based",
            "description": "Learning through building projects and practical exercises",
            "prompt_addition": "Structure each module around hands-on projects and exercises. Ensure learners build something tangible in each module."
        },
        "4": {
            "name": "Quick & Practical",
            "description": "Fast-paced, focused on immediate practical skills",
            "prompt_addition": "Keep content concise and practical. Focus on essential skills needed to get started quickly. Minimize theory, maximize practical application."
        },
        "5": {
            "name": "Comprehensive & Deep",
            "description": "Thorough coverage with both theory and practice",
            "prompt_addition": "Provide comprehensive coverage balancing theory and practice. Include edge cases, best practices, and deep dives into complex topics."
        },
        "6": {
            "name": "Beginner-Friendly",
            "description": "Slow-paced with lots of explanations and support",
            "prompt_addition": "Use simple language, provide detailed explanations, include many examples, and assume no prior knowledge. Break down complex concepts into digestible parts."
        }
    }
    
    @staticmethod
    def display_options():
        """Display available learning style options."""
        print("=" * 80)
        print("SELECT YOUR LEARNING STYLE PREFERENCE")
        print("=" * 80)
        for key, value in UserPreferences.PREFERENCE_OPTIONS.items():
            print(f"{key}. {value['name']}")
            print(f"   {value['description']}")
            print()
    
    @staticmethod
    def get_user_preference(auto_select: str = None) -> dict:
        """
        Get user preference either interactively or auto-select.
        
        Args:
            auto_select: If provided, auto-select this option (for notebook automation)
            
        Returns:
            Dictionary containing preference details
        """
        if auto_select:
            choice = auto_select
        else:
            UserPreferences.display_options()
            choice = input("Enter your choice (1-6): ").strip()
        
        if choice in UserPreferences.PREFERENCE_OPTIONS:
            selected = UserPreferences.PREFERENCE_OPTIONS[choice]
            print(f"\n✓ Selected: {selected['name']}")
            return selected
        else:
            print(f"\n⚠ Invalid choice. Defaulting to 'Real-world Examples'")
            return UserPreferences.PREFERENCE_OPTIONS["1"]

### Two-Layer Syllabus Generation System

In [39]:
class SyllabusGenerator:
    """Two-layer system for generating and validating course syllabi."""
    
    def __init__(self, model_name: str = "phi3:mini", temperature: float = 0.7):
        """Initialize the syllabus generator with two layers."""
        self.model_name = model_name
        self.temperature = temperature
        
        # Layer 1: Syllabus Generator
        self.generator_llm = ChatOllama(model=model_name, temperature=temperature)
        self.structured_generator = self.generator_llm.with_structured_output(Syllabus)
        
        # Layer 2: Syllabus Reviewer (lower temperature for consistency)
        self.reviewer_llm = ChatOllama(model=model_name, temperature=0.3)
        self.structured_reviewer = self.reviewer_llm.with_structured_output(SyllabusReview)
        
    def _create_generator_prompt(self, user_preference: dict) -> ChatPromptTemplate:
        """Create the prompt for syllabus generation (Layer 1)."""
        system_message = f"""You are an expert curriculum designer creating a detailed course syllabus.

LEARNING STYLE PREFERENCE:
{user_preference['prompt_addition']}

SYLLABUS STRUCTURE REQUIREMENTS:
1. Create 3-8 modules that comprehensively cover the course
2. Each module should have 3-10 topics
3. Modules should progress logically from foundational to advanced concepts
4. Topics within a module should be related and build upon each other

TOPIC REQUIREMENTS:
- short_description: 1-2 sentences explaining what the user will learn (user-facing)
- detailed_description: Comprehensive description of the topic scope, key concepts, learning outcomes, and teaching approach (AI-facing, 3-5 sentences)
- estimated_duration_minutes: Realistic time estimate (typically 15-120 minutes per topic)

CONTENT QUALITY RULES:
- No overlapping content between topics or modules
- Each topic must be substantial and well-defined
- Ensure logical flow and prerequisites are respected
- Total course should be comprehensive but not overwhelming
- Calculate total_estimated_hours accurately based on all topic durations

AVOID:
- Vague or generic topic descriptions
- Redundant content
- Unrealistic time estimates
- Missing key concepts for the course level"""

        user_message = """Create a detailed syllabus for the following course:

Course Name: {course_name}
Course Description: {course_description}
Difficulty Level: {difficulty}

Generate a complete, well-structured syllabus following all requirements."""

        return ChatPromptTemplate.from_messages([
            ("system", system_message),
            ("user", user_message)
        ])
    
    def _create_reviewer_prompt(self) -> ChatPromptTemplate:
        """Create the prompt for syllabus review (Layer 2)."""
        system_message = """You are a senior educational quality assurance expert reviewing course syllabi.

Your task is to critically evaluate syllabi for quality, coherence, and educational value.

EVALUATION CRITERIA:

1. STRUCTURE (Critical):
   - Are modules logically organized and progressive?
   - Do topics within modules relate to each other?
   - Is there a clear learning path from beginner to advanced?

2. CONTENT QUALITY (Critical):
   - Are topic descriptions clear and specific?
   - Do detailed descriptions provide enough context for content creation?
   - Is content appropriate for the stated difficulty level?

3. NO REDUNDANCY (Critical):
   - Are there overlapping topics?
   - Is any content repeated across modules?
   - Are topics distinct and well-defined?

4. COMPLETENESS (Critical):
   - Does the syllabus cover all essential aspects of the course?
   - Are there any obvious gaps in coverage?
   - Are prerequisites and dependencies clear?

5. REALISM (Important):
   - Are time estimates realistic?
   - Is the total course length appropriate?
   - Can topics be reasonably covered in the estimated time?

DECISION RULES:
- APPROVE if: All critical criteria are met and the syllabus is high quality
- REJECT if: Any critical criteria fail or there are major quality issues

When REJECTING:
- Provide specific, actionable feedback
- List all issues found
- Explain what needs to be fixed

When APPROVING:
- Provide positive feedback
- May suggest minor improvements (but still approve)"""

        user_message = """Review the following syllabus:

{syllabus_json}

Evaluate based on all criteria and decide whether to approve or reject."""

        return ChatPromptTemplate.from_messages([
            ("system", system_message),
            ("user", user_message)
        ])
    
    def generate_syllabus(self, course: Course, user_preference: dict, max_attempts: int = 3) -> tuple[Syllabus, list]:
        """
        Generate a syllabus with two-layer validation.
        
        Args:
            course: Course object to create syllabus for
            user_preference: User's learning style preference
            max_attempts: Maximum number of generation attempts
            
        Returns:
            Tuple of (approved_syllabus, attempt_history)
        """
        generator_prompt = self._create_generator_prompt(user_preference)
        generator_chain = generator_prompt | self.structured_generator
        
        reviewer_prompt = self._create_reviewer_prompt()
        reviewer_chain = reviewer_prompt | self.structured_reviewer
        
        attempt_history = []
        
        print("=" * 80)
        print(f"GENERATING SYLLABUS FOR: {course.course_name}")
        print(f"Learning Style: {user_preference['name']}")
        print("=" * 80)
        
        for attempt in range(1, max_attempts + 1):
            print(f"\n📝 Attempt {attempt}/{max_attempts}: Generating syllabus...")
            
            # Layer 1: Generate syllabus
            syllabus = generator_chain.invoke({
                "course_name": course.course_name,
                "course_description": course.description,
                "difficulty": course.difficulty
            })
            
            # Layer 2: Review syllabus
            print(f"🔍 Attempt {attempt}/{max_attempts}: Reviewing syllabus...")
            review = reviewer_chain.invoke({
                "syllabus_json": syllabus.model_dump_json(indent=2)
            })
            
            attempt_info = {
                "attempt": attempt,
                "syllabus": syllabus,
                "review": review
            }
            attempt_history.append(attempt_info)
            
            if review.approved:
                print(f"\n✅ APPROVED on attempt {attempt}!")
                print(f"Feedback: {review.feedback}")
                return syllabus, attempt_history
            else:
                print(f"\n❌ REJECTED on attempt {attempt}")
                print(f"Feedback: {review.feedback}")
                if review.issues:
                    print(f"Issues found:")
                    for issue in review.issues:
                        print(f"  - {issue}")
                
                if attempt < max_attempts:
                    print(f"\n🔄 Regenerating with feedback...")
        
        # If we get here, all attempts failed
        print(f"\n⚠️  WARNING: Syllabus not approved after {max_attempts} attempts.")
        print(f"Returning the last generated syllabus (with issues).")
        return syllabus, attempt_history
    
    def print_syllabus(self, syllabus: Syllabus) -> None:
        """Pretty print a syllabus."""
        print("\n" + "=" * 80)
        print(f"COURSE SYLLABUS: {syllabus.course_name}")
        print("=" * 80)
        print(f"Objective: {syllabus.course_objective}")
        print(f"Learning Style: {syllabus.learning_style}")
        print(f"Total Modules: {syllabus.total_modules}")
        print(f"Total Estimated Hours: {syllabus.total_estimated_hours:.1f}")
        print("=" * 80)
        
        for mod_idx, module in enumerate(syllabus.modules, 1):
            print(f"\n📚 MODULE {mod_idx}: {module.module_name}")
            print(f"   {module.module_description}")
            print(f"   Topics: {len(module.topics)}")
            print()
            
            for topic_idx, topic in enumerate(module.topics, 1):
                print(f"   {mod_idx}.{topic_idx} {topic.topic_name}")
                print(f"       📖 User Description: {topic.short_description}")
                print(f"       🤖 AI Description: {topic.detailed_description}")
                print(f"       ⏱️  Duration: {topic.estimated_duration_minutes} minutes")
                print()
        
        print("=" * 80)

### Example: Generate Syllabus for First Course

Let's take the first course from a broad topic and generate its syllabus.

In [40]:
# First, generate a course plan
sample_course_plan = planner.create_course_plan(
    course_title="Data Science",
    course_description="Learn data science from fundamentals to advanced applications including statistics, programming, machine learning, and data visualization."
)

planner.print_course_plan(sample_course_plan)

# Get the first course
first_course = sample_course_plan.courses[0]
print(f"\n🎯 Selected Course: {first_course.course_name}")
print(f"   Difficulty: {first_course.difficulty}")
print(f"   Description: {first_course.description}")

COURSE PLAN ANALYSIS
Topic is: BROAD
Total Courses: 5

📚 COURSE 1: Data Science Foundations
   Difficulty: Beginner
   Description: Explore the basics of data science: statistics, programming fundamentals (Python), and understanding different types of data. Introduction to basic libraries such as pandas and NumPy.
   Prerequisites: None

📚 COURSE 2: Statistics for Data Science
   Difficulty: Intermediate
   Description: Dive into statistics with a focus on the role of statistical analysis in data science. Learn hypothesis testing, confidence intervals, and p-values to make informed decisions using real datasets.
   Prerequisites: Data Science Foundations

📚 COURSE 3: Programming for Data Analysis
   Difficulty: Intermediate
   Description: Enhance your Python programming skills with data analysis libraries. Learn to manipulate and visualize large datasets effectively using tools like matplotlib, seaborn, Plotly.
   Prerequisites: Data Science Foundations

📚 COURSE 4: Machine Learning E

### Select Learning Style Preference

In [41]:
# Display options and get user preference
# For notebook automation, we auto-select option "1" (Real-world Examples)
# In interactive mode, remove the auto_select parameter

user_pref = UserPreferences.get_user_preference(auto_select="1")  # Change to None for interactive


✓ Selected: Real-world Examples


### Generate Syllabus with Two-Layer Validation

In [42]:
# Initialize the syllabus generator
syllabus_gen = SyllabusGenerator(model_name="phi3:mini", temperature=0.7)

# Generate syllabus with validation
approved_syllabus, history = syllabus_gen.generate_syllabus(
    course=first_course,
    user_preference=user_pref,
    max_attempts=3
)

# Display the final syllabus
syllabus_gen.print_syllabus(approved_syllabus)

GENERATING SYLLABUS FOR: Data Science Foundations
Learning Style: Real-world Examples

📝 Attempt 1/3: Generating syllabus...
🔍 Attempt 1/3: Reviewing syllabus...

❌ REJECTED on attempt 1
Feedback: While the syllabus is comprehensive in covering a broad range of topics within data science foundations with appropriate time allocation for each module, there are areas that need improvement to meet all critical criteria effectively. Here's what needs attention:
Issues found:
  - - The 'Data Processing' and 'Machine Learning Basics' modules overlap significantly in content related to Python programming basics which should be consolidated into a single module or clearly differentiated within the syllabus.
  - - There is no explicit mention of data ethics across all relevant topics, particularly missing from Data Science Tools where it could have been integrated as an important aspect alongside technical skills like using pandas and NumPy.
  - - The 'Data Processing' module lacks a topic on wo

### Review Attempt History

In [43]:
# Display the history of all attempts
print("=" * 80)
print("SYLLABUS GENERATION ATTEMPT HISTORY")
print("=" * 80)

for attempt_info in history:
    attempt = attempt_info['attempt']
    review = attempt_info['review']
    syllabus = attempt_info['syllabus']
    
    status = "✅ APPROVED" if review.approved else "❌ REJECTED"
    print(f"\nAttempt {attempt}: {status}")
    print(f"Modules Generated: {syllabus.total_modules}")
    print(f"Total Topics: {sum(len(m.topics) for m in syllabus.modules)}")
    print(f"Estimated Hours: {syllabus.total_estimated_hours:.1f}")
    print(f"Review Feedback: {review.feedback}")
    
    if review.issues:
        print(f"Issues Found:")
        for issue in review.issues:
            print(f"  - {issue}")
    print("-" * 80)

SYLLABUS GENERATION ATTEMPT HISTORY

Attempt 1: ❌ REJECTED
Modules Generated: 5
Total Topics: 31
Estimated Hours: 675.0
Review Feedback: While the syllabus is comprehensive in covering a broad range of topics within data science foundations with appropriate time allocation for each module, there are areas that need improvement to meet all critical criteria effectively. Here's what needs attention:
Issues Found:
  - - The 'Data Processing' and 'Machine Learning Basics' modules overlap significantly in content related to Python programming basics which should be consolidated into a single module or clearly differentiated within the syllabus.
  - - There is no explicit mention of data ethics across all relevant topics, particularly missing from Data Science Tools where it could have been integrated as an important aspect alongside technical skills like using pandas and NumPy.
  - - The 'Data Processing' module lacks a topic on working with categorical variables which are essential in prep

### Export Syllabus to JSON

In [44]:
# Export the approved syllabus to JSON
syllabus_json = approved_syllabus.model_dump_json(indent=2)

print("SYLLABUS JSON OUTPUT:")
print("=" * 80)
print(syllabus_json)
print("=" * 80)

# Save to file
import os

output_dir = "output"
os.makedirs(output_dir, exist_ok=True)

filename = f"{output_dir}/{approved_syllabus.course_name.replace(' ', '_')}_syllabus.json"
with open(filename, 'w', encoding='utf-8') as f:
    f.write(syllabus_json)

print(f"\n✓ Syllabus saved to: {filename}")

SYLLABUS JSON OUTPUT:
{
  "course_name": "Data Science Foundations",
  "course_objective": "This course introduces beginners to the fundamentals of data science with an emphasis on statistics, programming in Python for data analysis, and understanding various types of data.",
  "learning_style": "Concepts are taught through extensive real-world examples, case studies, practical applications, alongside interactive sessions. The course is designed to bridge theory with industry practices.",
  "total_modules": 3,
  "modules": [
    {
      "module_name": "Module 1: Introduction to Data Science",
      "module_description": "This module introduces the basics of data science and its relevance in various industries. Students will learn about different types of data through hands-on exercises.",
      "topics": [
        {
          "topic_name": "Understanding Data Science",
          "short_description": "Define what is meant by 'Data Science' and explore its applications in various industr

### Complete Pipeline Example

Here's a complete end-to-end example from topic to approved syllabus:

In [45]:
def complete_pipeline(topic_title: str, topic_description: str, learning_style: str = "1"):
    """
    Complete pipeline from topic to approved syllabus.
    
    Args:
        topic_title: The topic to create courses for
        topic_description: Description of the topic
        learning_style: Learning style preference (1-6)
    
    Returns:
        Tuple of (course_plan, first_course, syllabus, history)
    """
    print("🚀 COMPLETE COURSE GENERATION PIPELINE")
    print("=" * 80)
    
    # Step 1: Generate course plan
    print("\n📋 Step 1: Generating Course Plan...")
    course_plan = planner.create_course_plan(topic_title, topic_description)
    planner.print_course_plan(course_plan)
    
    # Step 2: Select first course
    first_course = course_plan.courses[0]
    print(f"\n🎯 Step 2: Selected First Course - {first_course.course_name}")
    
    # Step 3: Get user preferences
    print(f"\n🎨 Step 3: Applying Learning Style Preferences...")
    user_pref = UserPreferences.get_user_preference(auto_select=learning_style)
    
    # Step 4: Generate and validate syllabus
    print(f"\n📝 Step 4: Generating Syllabus with Two-Layer Validation...")
    syllabus, history = syllabus_gen.generate_syllabus(
        course=first_course,
        user_preference=user_pref,
        max_attempts=3
    )
    
    # Step 5: Display final result
    syllabus_gen.print_syllabus(syllabus)
    
    return course_plan, first_course, syllabus, history


# Example: Complete pipeline for Python Programming
python_plan, python_course, python_syllabus, python_history = complete_pipeline(
    topic_title="Python Programming",
    topic_description="Learn Python programming from basics to advanced concepts including data structures, OOP, and practical applications.",
    learning_style="1"  # Real-world Examples
)

🚀 COMPLETE COURSE GENERATION PIPELINE

📋 Step 1: Generating Course Plan...
COURSE PLAN ANALYSIS
Topic is: BROAD
Total Courses: 4

📚 COURSE 1: Python Fundamentals
   Difficulty: Beginner
   Description: Introduction to Python syntax and basic concepts including data types, variables, control structures, functions, and error handling. Understand the basics of file I/O operations.
   Prerequisites: None

📚 COURSE 2: Intermediate Python
   Difficulty: Intermediate
   Description: Building upon basic knowledge from 'Python Fundamentals', explore more complex data structures, classes and object-oriented programming (OOP), exception handling, and standard library modules.
   Prerequisites: None

📚 COURSE 3: Advanced Python
   Difficulty: Intermediate
   Description: Dive deeper into advanced features of the language such as decorators, generators, context managers, asynchronous programming with `asyncio`, and interaction with databases.
   Prerequisites: None

📚 COURSE 4: Python for Data Scie

### Summary: Two-Layer System Benefits

The two-layer validation system provides:

1. **Quality Assurance**: Second layer catches issues like:
   - Overlapping content
   - Incomplete coverage
   - Vague descriptions
   - Unrealistic time estimates

2. **Hallucination Prevention**: 
   - Detailed descriptions for AI ensure accurate content generation
   - Review layer validates factual accuracy and coherence

3. **Iterative Improvement**:
   - Automatic regeneration with feedback
   - Up to N attempts to get it right
   - Clear tracking of issues and improvements

4. **Flexibility**:
   - Multiple learning style options
   - Customizable prompts per preference
   - Variable module/topic structure

5. **Production Ready**:
   - Complete error handling
   - JSON export for API integration
   - Detailed logging and history tracking